<a href="https://colab.research.google.com/github/Anughna04/Credit-Card-Fraud-detection/blob/main/frauddetectionmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,accuracy_score,roc_curve,auc
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
import pickle

#import and read the dataset
df=pd.read_csv('creditcard.csv')
df.head()

#Standardize and scale the features
scaler=StandardScaler()
df.iloc[:,:-1]=scaler.fit_transform(df.iloc[:,:-1])

#divide dataset into features and target
x=df.drop("Class",axis=1)
y=df["Class"]
print(y.value_counts())

#Use smote to handle bias
smote=SMOTE(sampling_strategy=0.33,random_state=42)
x_resampled,y_resampled=smote.fit_resample(x,y)
print(y_resampled.value_counts())

#split into training and testing sets
x_train,x_test,y_train,y_test=train_test_split(x_resampled,y_resampled,test_size=0.20,random_state=42)

#define the model
model=LogisticRegression()

#cross validation with stratified kfold method
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, x_train, y_train, cv=skf, scoring="f1")
print(f"Mean F1-score across 5 folds: {cv_scores.mean():.4f}")

#train the model
model.fit(x_train,y_train)

#evaluate model performance
y_pred=model.predict(x_test)
accuracy=accuracy_score(y_test,y_pred)
print("Accuracy:",accuracy)

"""classification report is considered more than accuracy,as it gives info about key metrics
like precision and recall for imbalanced data"""
print(classification_report(y_test,y_pred))

"""The ROC curve helps evaluate the model’s trade-off between detecting fraud(high recall)
and avoiding false alarms(low FPR)"""
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

#plot the roc-auc curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color="blue", label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color="gray", linestyle="--")  # Dashed diagonal line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC-AUC Curve for Fraud Detection")
plt.legend(loc="lower right")
plt.show()

#save the model into a pickle file
pickle.dump(model,open('model.pkl','wb'))